In [1]:
from sklearn import svm as sv
import pandas as pd
import pickle
import chess
import numpy as np
#from __future__ import with_statement
import datetime
import time
import os
from pathlib import Path

class svm():
    
    def __init__(self, filename = None, historic = False, gam = 1/69):
        if filename is None:
                self._model = sv.SVC(kernel = 'rbf', gamma=gam, probability = True)


        elif filename is not None and historic == True:
            #self._model = svm.SVR(kernel = 'rbf')
            try:
                with open(filename, 'rb') as file:
                    self._model = pickle.load(file)
            except EnvironmentError as e:
                print(e)
                
                
        # placeholder!!!
        else: 
            self._model = sv.SVC(kernel = 'rbf', gamma=gam)
      
    
    def fit(self, dataset, formatted = True):
        y = dataset.pop('class')
        self._model.fit(dataset,y)
    
    def predict(self, dataset):
        try:
            return self._model.predict(dataset)
        except Exception as e:
            print(e)
            return False


    def predict_proba(self, dataset, formatted = True):
        try:
             return float(self._model.predict_proba(dataset)[:, np.where(self._model.classes_ == 1)].item(0))
        except Exception as e:
            return 0.5
    
    def write_file(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self._model, file)
    
    def load_file(self, filename):
        with open(filename, 'rb') as file:
            self._model = pickle.load(file)

In [2]:
data = pd.read_csv('../src/models/history/history.csv')
data.head()

,state,move,class
0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,g1f3,1
1,rnbqkbnr/pppp1ppp/8/4p3/8/5N2/PPPPPPPP/RNBQKB1...,f3e5,1
2,rnbqkbnr/ppp2ppp/8/3pN3/8/8/PPPPPPPP/RNBQKB1R ...,b1c3,1
3,rnbqkb1r/ppp1nppp/8/3pN3/8/2N5/PPPPPPPP/R1BQKB...,e2e3,1
4,rnbqkbr1/ppp1nppp/8/3pN3/8/2N1P3/PPPP1PPP/R1BQ...,d1h5,1


In [3]:
class preprocessor(object):

    def __init__(self):
        super().__init__

    def fit(self, filename = None, raw_data = None):
        if filename is not None:
            self.raw_data = pd.read_csv(filename)
        elif raw_data is not None:
            self.raw_data = pd.DataFrame(raw_data)
        return self.raw_data

    def transform(self, predict = False):
        data = self.raw_data
        print(data)
        if not predict:
            class_ = data.pop('class')
        data['state'] = data['state'].apply(self.serialize2)
        data = pd.concat([data['state'].apply(pd.Series), data['move']], axis = 1)
        move = data.pop('move')
        move = move.apply(self.listify).apply(pd.Series)

        #data = pd.concat([data,move.apply(self.listify).apply(pd.Series)], axis = 1)
        move = pd.DataFrame(move).rename(columns = {0:'m1',1:'m2',2:'m3',3:'m4',4:'m5'})

        try:
            move['m5'] = move['m5'].apply(self.encode_move)
        except Exception:
            move['m5'] = 0

        move['m1'] = move['m1'].apply(self.encode_move)
        move['m2'] = move['m2'].apply(self.encode_move)
        move['m3'] = move['m3'].apply(self.encode_move)
        move['m4'] = move['m4'].apply(self.encode_move)

        
        data = pd.concat([data,move], axis = 1)
        if not predict:
            data = pd.concat([data,class_], axis = 1)
        return data
    
    def encode_move(self,m):
        try:
            val = ord(m)
            return val
        except Exception:
            return 0



    def listify(self,string):
        return list(str(string))
        
    def bin_matrix_to_decimal(self, state):
        new_state = list()
        ret_state = list()
        for i in range(0,8):
            for j in range(0,8):
                new_state.append((np.array([state[0][i][j],state[1][i][j],state[2][i][j],state[3][i][j]])))
        for i in range(64):
            ret_state.append(int(np.packbits(new_state[i])))
        return ret_state
            

    def serialize(self,board):
        import numpy as np
        assert board.is_valid()

        bstate = np.zeros(64, np.uint8)
        for i in range(64):
            pp = board.piece_at(i)
            if pp is not None:
                #print(i, pp.symbol())
                bstate[i] = {"P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6, \
                        "p": 9, "n":10, "b":11, "r":12, "q":13, "k": 14}[pp.symbol()]
        if board.has_queenside_castling_rights(chess.WHITE):
            assert bstate[0] == 4
            bstate[0] = 7
        if board.has_kingside_castling_rights(chess.WHITE):
            assert bstate[7] == 4
            bstate[7] = 7
        if board.has_queenside_castling_rights(chess.BLACK):
            assert bstate[56] == 8+4
            bstate[56] = 8+7
        if board.has_kingside_castling_rights(chess.BLACK):
            assert bstate[63] == 8+4
            bstate[63] = 8+7

        if board.ep_square is not None:
            assert bstate[board.ep_square] == 0
            bstate[board.ep_square] = 8
        bstate = bstate.reshape(8,8)

        state = np.zeros((4,8,8), np.uint8)
        
        state[0] = (bstate>>3)&1
        state[1] = (bstate>>2)&1
        state[2] = (bstate>>1)&1
        state[3] = (bstate>>0)&1
        """
        state.append(self.bin_matrix_to_decimal((bstate>>3)&1))
        state.append(self.bin_matrix_to_decimal((bstate>>2)&1))
        state.append(self.bin_matrix_to_decimal((bstate>>1)&1))
        state.append(self.bin_matrix_to_decimal((bstate>>0)&1))
        """

        new_state = self.bin_matrix_to_decimal(state)
        new_state.append(board.turn*1.0)
    
        return new_state

    def serialize2(self,board):
        return self.serialize(chess.Board(str(board)))


In [4]:
prec = preprocessor()
prec.fit(filename = '../src/models/history/history.csv')
data = prec.transform()
data.head()

                                                 state  move  class
0    rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...  g1f3      1
1    rnbqkbnr/pppp1ppp/8/4p3/8/5N2/PPPPPPPP/RNBQKB1...  f3e5      1
2    rnbqkbnr/ppp2ppp/8/3pN3/8/8/PPPPPPPP/RNBQKB1R ...  b1c3      1
3    rnbqkb1r/ppp1nppp/8/3pN3/8/2N5/PPPPPPPP/R1BQKB...  e2e3      1
4    rnbqkbr1/ppp1nppp/8/3pN3/8/2N1P3/PPPP1PPP/R1BQ...  d1h5      1
..                                                 ...   ...    ...
955  4k1nr/2p5/B1P1p3/3pP2p/1B1P1p1P/p1P2N2/P3NPP1/...  e8f7      0
956  6nr/2p2k2/B1P1p3/3pP2p/1B1P1N1P/p1P2N2/P4PP1/R...  f7g7      0
957  6nr/2p3k1/B1P1N3/3pP2p/1B1P3P/p1P2N2/P4PP1/R2Q...  g7g6      0
958  6nr/2p5/B1P1N1k1/3pP2p/1B1P3P/p1PQ1N2/P4PP1/R3...  g6f7      0
959  6nr/2p2k2/B1P1N3/3pP1Np/1B1P3P/p1PQ4/P4PP1/R3R...  f7e8      0

[960 rows x 3 columns]


,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,m1,m2,m3,m4,m5,class
0,112.0,32.0,48.0,80.0,96.0,48.0,32.0,112.0,16.0,16.0,...,176.0,160.0,240.0,1.0,103,49,102,51,0,1
1,112.0,32.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,176.0,160.0,240.0,1.0,102,51,101,53,0,1
2,112.0,32.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,176.0,160.0,240.0,1.0,98,49,99,51,0,1
3,112.0,0.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,176.0,0.0,240.0,1.0,101,50,101,51,0,1
4,112.0,0.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,176.0,192.0,0.0,1.0,100,49,104,53,0,1


In [5]:
model = svm()
model.fit(data)

In [6]:
 model.write_file('test_model_1.pkl')

In [8]:
new_data = [{'state':'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1','move':'g1f3'}]
prec = preprocessor()
prec.fit(raw_data = new_data)
new_data = prec.transform(predict = True)
print(model.predict_proba(new_data))

                                               state  move
0  rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...  g1f3
0.9971541640148628
